In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
areas=avisos['nombre_area'].value_counts()
avisos['nombre_area']=avisos['nombre_area'].replace(areas.index, areas.values)

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral','nombre_area']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')
avisos_no_one_hot['nombre_area']=avisos_no_one_hot['nombre_area'].astype('category')
avisos_no_one_hot=avisos_no_one_hot.rename(columns={'nombre_area': 'nombre_area_aux'})

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501973


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad
9946093,1112501311,YjVVYXJ,0,30.0,True,False,False,False,False,False,...,False,False,False,False,True,False,Part-time,Otro,Educación/ Docentes,NaN
3571511,1112368321,3N2pWWX,0,20.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Soporte Técnico,NaN
8417256,1112409173,lDLaYlX,0,200.0,False,False,True,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Soporte Técnico,NaN
12252527,1112421526,LNKrlBB,0,23.0,False,True,False,False,False,False,...,False,False,True,False,False,False,Teletrabajo,Jefe / Supervisor / Responsable,Comercial,NaN
8101050,1112426825,akRpX2E,1,24.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Otras Especialidades médicas,0.006270
12461306,1112289648,akDpzpJ,1,23.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Administración de Personal,NaN
8103938,1112401409,8m2LvR,1,35.0,True,False,False,False,False,False,...,False,False,False,True,False,False,Full-time,Junior,Compras,0.008264
343920,1112402352,kPjb4ma,0,20.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,NaN
15822150,1112513389,8XxKMl,0,68.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Soporte Técnico,NaN
8870012,1112373764,8A0NMW,0,29.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Temporario,Senior / Semi-Senior,Producción,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad
13594293,1112283002,Rzr4zB1,1,200.0,False,False,True,False,False,False,...,False,False,False,False,True,False,Full-time,Otro,Salud,0.000000
7015032,1112425062,Yd8qq1,1,28.0,True,False,False,False,False,False,...,False,False,False,False,False,False,Full-time,-,Oficios y Profesiones,0.000000
6860339,1112299161,2zm4Wv1,0,28.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Producción,0.000000
11886238,1112310342,akjvmDq,0,20.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Comercial,0.000000
16031792,1112463922,6AOMMr,0,33.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Programación,0.000000
13115641,1112201811,owaG56r,0,31.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Part-time,Senior / Semi-Senior,Tecnologia / Sistemas,0.000000
12824781,1112483445,VNl40Xm,0,23.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Programación,0.000000
8279130,1112341385,vVj5B4x,1,31.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Gastronomia,0.000000
7821014,1112141105,mzjjY69,0,27.0,True,False,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,0.000000
15257754,1112292676,6jjao4,0,29.0,False,True,False,False,False,False,...,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,Programación,0.000000


In [15]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area_aux'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area_aux,postulaciones_misma_area
0,0z5Dmrd,Administración,1
1,0z5Dmrd,Administración de Personal,1
2,0z5Dmrd,Almacén / Depósito / Expedición,2
3,0z5Dmrd,Atención al Cliente,1
4,0z5Dmrd,Auditoría,1


In [16]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
2,0z5Dmrd,Junior,4
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,19


In [17]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,24
1,0z5Dmrd,Part-time,1
2,0z5Dmrd,Teletrabajo,1
3,0z5JW1r,Full-time,26
4,0z5JW1r,Part-time,1


In [18]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
7569875,1112267946,1DJ0Pz,1,27.0,True,False,False,False,False,False,...,False,True,False,Full-time,Otro,Producción,0.000000,0,45,3
7020067,1112256865,owaR4zZ,0,27.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ventas,0.000000,3,20,14
10147573,1112485611,e4GOkr,0,33.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Infraestructura,0.000000,0,18,13
7994002,1112263288,YalrXD,0,27.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,0.000000,0,16,12
10863050,1112247187,xkdOEQ0,1,55.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,0.000000,0,44,34
11451145,1112471405,YRkOAX,0,30.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Atención al Cliente,0.000000,1,58,41
15199584,1112297025,4rPjBqK,1,30.0,False,True,False,False,False,False,...,False,True,False,Full-time,Otro,Ingeniería Oficina Técnica / Proyecto,0.000000,3,101,4
14505774,1112243832,8MPQEx3,0,28.0,True,False,False,False,False,False,...,True,False,False,Full-time,Junior,Producción,0.000000,2,19,2
7308897,1112347217,ZDPmRQ1,0,39.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Contabilidad,0.000000,3,17,16
1700661,1112422003,2zwm9vQ,1,28.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Finanzas,0.025316,13,53,7


In [19]:
train, test=train_test_split(postulaciones.sample(2*(len(postulaciones)//3)),test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')
features.remove('nombre_area_aux')
features.remove('tipo_de_trabajo')
features.remove('nivel_laboral')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  10187895 Test:  536205


In [20]:
c=0
gc.collect()

95

In [21]:
c=XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.01,subsample=0.8,colsample_bytree=0.8,n_estimators=400,silent=False,n_jobs=-1,verbose=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")
dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=10)


[0]	validation_0-error:0.100879
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.094268
[2]	validation_0-error:0.092049
[3]	validation_0-error:0.09092
[4]	validation_0-error:0.091564
[5]	validation_0-error:0.091909
[6]	validation_0-error:0.089874
[7]	validation_0-error:0.090538
[8]	validation_0-error:0.090204
[9]	validation_0-error:0.090865
[10]	validation_0-error:0.0904
[11]	validation_0-error:0.09035
[12]	validation_0-error:0.090294
[13]	validation_0-error:0.090747
[14]	validation_0-error:0.090077
[15]	validation_0-error:0.089947
[16]	validation_0-error:0.089656
[17]	validation_0-error:0.08963
[18]	validation_0-error:0.089451
[19]	validation_0-error:0.089164
[20]	validation_0-error:0.08905
[21]	validation_0-error:0.089089
[22]	validation_0-error:0.08902
[23]	validation_0-error:0.089117
[24]	validation_0-error:0.089048
[25]	validation_0-error:0.089294
[26]	validation_0-error:0.089341
[27]	validation_0-error:0.089283
[28]	validation_0-error:0.0

In [22]:
score=c.score(x_test,y_test)*100
print(score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


91.07803918277526


In [23]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/XGBoost v4.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/XGBoost v4.pkl']

In [24]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
97461,97218,1112498755,8VQGXL,64.0,True,False,False,False,False,False,...,False,True,False,Full-time,Otro,Ventas,0.000000,1.0,17.0,2.0
32186,13761,1112054318,1apbJN,38.0,True,False,False,False,False,False,...,False,False,False,Full-time,-,Comercial,0.000000,0.0,23.0,0.0
9665,75655,1112464172,vV3ZAD1,24.0,True,False,False,False,False,False,...,True,False,False,Full-time,Junior,Selección,0.000000,0.0,18.0,6.0
31643,10967,1111933464,EPOlj0,35.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ingeniería de Procesos,0.000000,0.0,18.0,14.0
79875,88030,1112466951,owEYW4W,25.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Comercial,0.034483,6.0,29.0,24.0
58930,7671,1111728385,8RRvEW,43.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Administración,0.000000,1.0,22.0,2.0
84450,33218,1112401076,epw33o,35.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Capacitación,0.000000,0.0,21.0,15.0
66447,12566,1112011933,eLZGLE,29.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Laboratorio,0.000000,0.0,19.0,17.0
51346,76969,1112464276,ak4DRd9,19.0,True,False,False,False,False,False,...,False,False,False,Part-time,-,Telemarketing,0.003185,11.0,84.0,11.0
15553,66540,1112459934,ZDrAYjZ,25.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Recepcionista,0.000000,0.0,20.0,12.0


In [25]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.63626504, 0.36373493],
       [0.512385  , 0.48761496],
       [0.6256599 , 0.37434012],
       ...,
       [0.63166684, 0.36833316],
       [0.6316451 , 0.3683549 ],
       [0.6350863 , 0.3649137 ]], dtype=float32)

In [26]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [27]:
prediccion['sepostulo']=y_final2

In [28]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,False,False,Full-time,Jefe / Supervisor / Responsable,Comercial,0.0,0.0,16.0,0.0,0.363735
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,False,False,Full-time,Jefe / Supervisor / Responsable,Comercial,0.0,5.0,62.0,0.0,0.487615
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,False,False,Full-time,Jefe / Supervisor / Responsable,Comercial,0.0,3.0,16.0,1.0,0.374340
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,True,False,Full-time,Otro,Tecnologia / Sistemas,0.0,0.0,21.0,0.0,0.368996
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,True,False,Full-time,Otro,Tecnologia / Sistemas,0.0,1.0,15.0,2.0,0.361764


In [29]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.drop(['tipo_de_trabajo','nivel_laboral','nombre_area_aux'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.363735
1,1,0.487615
2,2,0.374340
3,3,0.368996
4,4,0.361764


In [30]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion38.csv",index=False)

In [31]:
prediccion.sepostulo.mean()

0.4867469890096784